In [38]:
!pip install PyPDF2 langchain openai sentence-transformers


In [1]:
import os
from PyPDF2 import PdfReader
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain, ConversationChain
from langchain.memory import ConversationBufferWindowMemory
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
import openai

# OpenAI API Key 설정
OPENAI_API_KEY = 'sk-'
openai.api_key = OPENAI_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# 텍스트 파일에서 텍스트 추출 함수
def extract_text_from_txt(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# PDF에서 텍스트 추출 함수
def extract_text_from_pdf(file_paths):
    text = ""
    for file_path in file_paths:
        pdf_reader = PdfReader(file_path)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

# 텍스트를 청크 단위로 분할 및 벡터 스토어 생성 함수
def process_documents(text, embeddings):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    text_chunks = text_splitter.split_text(text)
    return FAISS.from_texts(text_chunks, embeddings)

# 문서 기반 대화 체인 생성 함수
def get_document_conversation_chain(vectorstore):
    memory = ConversationBufferWindowMemory(memory_key='chat_history', return_messages=True, window_size=10)
    return ConversationalRetrievalChain.from_llm(
        llm=ChatOpenAI(temperature=0, model_name='gpt-4'),
        retriever=vectorstore.as_retriever(),
        memory=memory
    )

# 일반 대화 체인 생성 함수
def get_general_conversation_chain():
    memory = ConversationBufferWindowMemory(memory_key='history', return_messages=True, window_size=10)
    return ConversationChain(llm=ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo'), memory=memory, input_key="input", output_key="output")


In [6]:
def run_document_chatbot():
    file_type = input("파일 유형을 선택하세요 (txt/pdf): ").strip().lower()
    file_paths = input("파일 경로를 입력하세요 (여러 개일 경우 쉼표로 구분): ").strip().split(',')

    all_text = ""
    if file_type == 'txt':
        for file_path in file_paths:
            all_text += extract_text_from_txt(file_path)
    elif file_type == 'pdf':
        all_text += extract_text_from_pdf(file_paths)

    if all_text.strip():  # 텍스트가 비어 있지 않은지 확인
        embeddings = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')
        vectorstore = process_documents(all_text, embeddings)

        if vectorstore:
            document_chain = get_document_conversation_chain(vectorstore)
            while True:
                user_input = input("질문을 입력하세요 ('quit' 입력 시 종료): ").strip()
                if user_input.lower() == 'quit':
                    break
                result = document_chain({"question": user_input})
                response = result.get("answer", "응답을 생성하는 동안 오류가 발생했습니다.")
                print(response)
    else:
        print("파일을 처리할 수 없거나 파일이 비어 있습니다. 대화형 챗봇으로 전환합니다.")
        run_general_chatbot()

def run_general_chatbot():
    conversation = get_general_conversation_chain()
    chat_history = []

    while True:
        user_input = input("질문을 입력하세요 ('quit' 입력 시 종료): ").strip()
        if user_input.lower() == 'quit':
            break
        chat_history.append({"role": "user", "content": user_input})
        result = conversation({"input": user_input})
        response = result.get("output", "응답을 생성하는 동안 오류가 발생했습니다.")
        chat_history.append({"role": "assistant", "content": response})
        print(response)
